<h1>Coastline change monitoring</h1>

<h5>Advice from Sachin Singh and Robbi Bishop-Taylor</h5>

<h1>Notes</h1>

A simplified version of the DEA Coastlines method that combines data from the Landsat 5, 7 and 8 satellites with image compositing and tide filtering techniques to accurately map shorelines across time and identify areas that have changed significantly between 1988 and 2018. This example demonstrates how to: 
1. Load cloud-free Landsat time series data
2. Compute a water index (MNDWI)
3. Filter images by tide height
4. Create “summary” or composite images for given time periods
5. Extract and visualise shorelines across time






<h4>STEP 1 - Load python packages</h4>


In [2]:
%matplotlib inline

import datacube
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard, mostcommon_crs
from dea_tools.plotting import rgb, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.spatial import subpixel_contours
from dea_tools.coastal import tidal_tag
from dea_tools.dask import create_local_dask_cluster

# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)

Client Scheduler: tcp://127.0.0.1:38855 Dashboard: /user/nmetherall/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 13.11 GB


The next step would usually involve connecting the Open Data Cube - however, in a Microsoft Planetary Computer, this step is not necessary.  

In [3]:
# dc = datacube.Datacube(app='Coastal_erosion')

To ensure that the tidal modelling part of this analysis works correctly, please make sure the centre of the study area is located over water when setting lat_range and lon_range.

<h4>STEP 2 - Set up parameters</h4>

The location, I have selected for this is Wotje Atoll in RMI. We can easily change the location to another case study by selecting a adjusting the lat and lon_range parameters. 

In [4]:
lat_range = (9.376, 9.571)
lon_range = (170.114, 170.278)
time_range = ('1990', '2020')
time_step = '1Y'
tide_range = (0.25, 0.75)

<h4>Display case study site</h4>

In [6]:
display_map(x=lon_range, y=lat_range)


<h4>STEP 3 - Load cloud-masked Landsat data</h4>
The first step in this analysis is to load in Landsat data. The load_ard function loads in data from the Landsat 5, 7 and 8 satellites for the area and time included in lat_range, lon_range and time_range. The function will also automatically mask out clouds from the dataset, allowing us to focus on pixels that contain useful data:

Note - that we are not able to follow this step outside of the datacube environment:

In [7]:
# Create the 'query' dictionary object, which contains the longitudes,
# latitudes and time provided above
query = {
    'y': lat_range,
    'x': lon_range,
    'time': time_range,
    'measurements': ['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_swir_1'],
    'resolution': (-30, 30),
}

# Identify the most common projection system in the input query
output_crs = mostcommon_crs(dc=dc, product='ga_ls5t_ard_3', query=query)

# Load available data from all three Landsat satellites
landsat_ds = load_ard(dc=dc,
                      products=['ga_ls5t_ard_3',
                                'ga_ls7e_ard_3',
                                'ga_ls8c_ard_3'],
                      output_crs=output_crs,
                      align=(15, 15),
                      group_by='solar_day',
                      dask_chunks={},
                      **query)


NameError: name 'dc' is not defined